In [1]:
from transformers import AutoTokenizer, MBartForConditionalGeneration
tokenizer = AutoTokenizer.from_pretrained('ku-nlp/bart-base-japanese')
model = MBartForConditionalGeneration.from_pretrained('ku-nlp/bart-base-japanese')

/home/is/kaifan-l/miniconda3/envs/summarization/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
本稿では実際に計算機上で試作した論説文要約システムに関して, これで用いられている論説文要約の手法の紹介と, これによって出力された文章の評価を行う.

In [8]:
sentence = '本稿 では 実際に 計算機上 で試作した 論説文 要約システム に 関して, これで 用 いられている 論説文 要約の 手法の 紹介と, これ によって 出力された 文章の 評価を 行う.'  # input should be segmented into words by Juman++ in advance
encoding = tokenizer(sentence, return_tensors='pt')

In [9]:
encoding

{'input_ids': tensor([[    0, 18573, 24285,  1367, 28430,  3126, 28626, 28536,    12, 28884,
         28540,   198, 23644, 28673, 21607, 21699,     7,  2210,     3,   180,
         28439,   235, 21911,   630, 28441, 28433, 23644, 28673, 21607, 28428,
          3713, 28428,  1972, 28436,     3,   180,     7, 28476,    44,  3331,
         28459,   483,  7118, 28428,  1418, 28437,   794,     3,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}

In [2]:
from pyknp import Juman

jumanpp = Juman()

text = '吾輩は猫である.'
mlist = jumanpp.analysis(text)

print([mrph.midasi for mrph in mlist.mrph_list()])
# ['吾輩', 'は', '猫', 'である', '。']

Exception: Can't find JUMAN command: jumanpp

In [3]:
import nltk
nltk.download('punkt')  # 下载标点符号模型

from nltk.tokenize import sent_tokenize

japanese_text = "日本語のテキストを分割するのに使える。 英語と同様、文単位に分割することが可能。"

sentences = sent_tokenize(japanese_text)
print(sentences)


['日本語のテキストを分割するのに使える。英語と同様、文単位に分割することが可能。']


[nltk_data] Downloading package punkt to
[nltk_data]     /home/is/kaifan-l/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained('ku-nlp/bart-base-japanese')
model = AutoModelForSeq2SeqLM.from_pretrained('ku-nlp/bart-base-japanese')
sentence = '京都 大学 で 自然 言語 処理 を 専攻 する 。'  # input should be segmented into words by Juman++ in advance
encoding = tokenizer(sentence, return_tensors='pt')


/home/is/kaifan-l/miniconda3/envs/summarization/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(32002, 768, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(32002, 768, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,)

In [2]:
import evaluate 

metric = evaluate.load('rouge')

In [40]:
metric.compute(predictions=['京都 大学 で を 専攻する。'], references=['京都 大学 で 自然言語 処理を専攻する。'],tokenizer=lambda x: x.split())

{'rouge1': 0.6, 'rouge2': 0.5, 'rougeL': 0.6, 'rougeLsum': 0.6}

In [34]:
from rouge import Rouge

rouge = Rouge()
predictions = ['日本語 は る 言語 である']
labels = ['日本語 は 日本 で 使われている 言語 である']

scores = rouge.get_scores(predictions, labels, avg=True)
scores


{'rouge-1': {'r': 0.5714285714285714, 'p': 0.8, 'f': 0.6666666618055556},
 'rouge-2': {'r': 0.3333333333333333, 'p': 0.5, 'f': 0.39999999520000007},
 'rouge-l': {'r': 0.5714285714285714, 'p': 0.8, 'f': 0.6666666618055556}}

In [29]:
scores


{'rouge-1': {'r': 0.5714285714285714, 'p': 0.8, 'f': 0.6666666618055556},
 'rouge-2': {'r': 0.3333333333333333, 'p': 0.5, 'f': 0.39999999520000007},
 'rouge-l': {'r': 0.5714285714285714, 'p': 0.8, 'f': 0.6666666618055556}}

In [21]:
from rouge import Rouge

# 参考摘要
reference_summary = "日本　摘要文本"

# 生成摘要
generated_summary = "日本　語　摘要文本"

rouge = Rouge()
scores = rouge.get_scores(generated_summary, reference_summary)

print(scores)


[{'rouge-1': {'r': 1.0, 'p': 0.6666666666666666, 'f': 0.7999999952000001}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 1.0, 'p': 0.6666666666666666, 'f': 0.7999999952000001}}]


In [47]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, AutoConfig
config = AutoConfig.from_pretrained('ku-nlp/bart-base-japanese')
training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    generation_config = AutoConfig.from_pretrained('ku-nlp/bart-base-japanese'), 
)


In [48]:
print(training_args.generation_config)

MBartConfig {
  "_name_or_path": "ku-nlp/bart-base-japanese",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 2,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL

In [4]:
from fugashi import GenericTagger
tagger = GenericTagger()

# parse can be used as normal
tagger.parse('something')
# features from the dictionary can be accessed by field numbers
for word in tagger(text):
    print(word.surface, word.feature[0])

RuntimeError: 
Failed initializing MeCab. Please see the README for possible solutions:

    https://github.com/polm/fugashi

If you are still having trouble, please file an issue here, and include the
ERROR DETAILS below:

    https://github.com/polm/fugashi/issues

issueを英語で書く必要はありません。

------------------- ERROR DETAILS ------------------------
arguments: [b'fugashi', b'-C']
param.cpp(69) [ifs] no such file or directory: /usr/local/etc/mecabrc
----------------------------------------------------------


In [34]:
import MeCab

def japanese_sent_tokenize(text):
    m = MeCab.Tagger('-Owakati')  # 初始化 MeCab 分词器
    sentences = []
    current_sentence = ""
    parsed = m.parse(text).split()  # 使用 MeCab 对文本进行分词
    print(parsed)
    for token in parsed:
        current_sentence += token
        if '。' in token:  # 根据句号来分割句子
            sentences.append(current_sentence)
            current_sentence = ""
    if current_sentence:
        sentences.append(current_sentence)
    return sentences

japanese_text = "日本語のテキストを分割するのに使える。英語と同様、文単位に分割することが可能。"

japanese_sentences = japanese_sent_tokenize(japanese_text)
print(japanese_sentences)


['日本', '語', 'の', 'テキスト', 'を', '分割', 'する', 'の', 'に', '使える', '。', '英語', 'と', '同様', '、', '文', '単位', 'に', '分割', 'する', 'こと', 'が', '可能', '。']
['日本語のテキストを分割するのに使える。', '英語と同様、文単位に分割することが可能。']


In [38]:
import re
import nltk

sent_detector = nltk.RegexpTokenizer(u'[^　！？。]*[！？。.\n]')

sents = sent_detector.tokenize(u"　原子番号９２のウランより重い元素は全て人工的に合成され、１１８番まで発見の報告がある？\
      １１３番については、理研と米露の共同チームがそれぞれ「発見した」と報告し、国際純正・応用化学連合と国際純粋・応用物理学連合の合同作業部会が審査していた! 両学会は「データの確実性が高い」ことを理由に、理研の発見を認定し、３１日に森田さんに通知した。未確定だった１１５番と１１７番、１１８番の新元素は米露チームの発見を認めた。森田さんは「周期表に名前が残ることは感慨深い。大勢の共同研究者にまずは感謝したい」と述べた。 \n")

print(sents)
for s in sents:
    print(s)

['原子番号９２のウランより重い元素は全て人工的に合成され、１１８番まで発見の報告がある？', '      １１３番については、理研と米露の共同チームがそれぞれ「発見した」と報告し、国際純正・応用化学連合と国際純粋・応用物理学連合の合同作業部会が審査していた! 両学会は「データの確実性が高い」ことを理由に、理研の発見を認定し、３１日に森田さんに通知した。', '未確定だった１１５番と１１７番、１１８番の新元素は米露チームの発見を認めた。', '森田さんは「周期表に名前が残ることは感慨深い。', '大勢の共同研究者にまずは感謝したい」と述べた。', ' \n']
原子番号９２のウランより重い元素は全て人工的に合成され、１１８番まで発見の報告がある？
      １１３番については、理研と米露の共同チームがそれぞれ「発見した」と報告し、国際純正・応用化学連合と国際純粋・応用物理学連合の合同作業部会が審査していた! 両学会は「データの確実性が高い」ことを理由に、理研の発見を認定し、３１日に森田さんに通知した。
未確定だった１１５番と１１７番、１１８番の新元素は米露チームの発見を認めた。
森田さんは「周期表に名前が残ることは感慨深い。
大勢の共同研究者にまずは感謝したい」と述べた。
 



In [3]:
import evaluate

metric = evaluate.load('rouge')

In [7]:
preds = ['i am a cat','i am a dog']
labels = ['i am a cat', 'am i a dog']

result = metric.compute(predictions=preds, references=labels, tokenizer=lambda x: x.split())

In [8]:
result

{'rouge1': 1.0,
 'rouge2': 0.6666666666666666,
 'rougeL': 0.875,
 'rougeLsum': 0.875}

In [6]:
import pandas as pd

df = pd.read_json("datasets/pubmed-dataset-incremental-final/test.json", lines=True)


FileNotFoundError: File datasets/pubmed-dataset-incremental-final/test.json does not exist

In [5]:
df['abstract_text'][0]

['objective: to evaluate the efficacy and safety of outpatient management of severe ovarian hyper stimulation syndrome( ohs s) requiring placement of a pigtail catheter.',
 'objective: to evaluate the efficacy and safety of outpatient management of severe ovarian hyper stimulation syndrome( ohs s) requiring placement of a pigtail catheter. methods: retrospective analysis of thirty- three consecutive patients who underwent in- vitro fertilization( 2003- 2009) and developed severe/ critical ohs s requiring placement of a pigtail catheter. patients who were managed on outpatient basis were monitored by frequent office visits, daily phone calls, and received iv normal saline for hydration when required.',
 'objective: to evaluate the efficacy and safety of outpatient management of severe ovarian hyper stimulation syndrome( ohs s) requiring placement of a pigtail catheter. methods: retrospective analysis of thirty- three consecutive patients who underwent in- vitro fertilization( 2003- 2009

In [9]:
import evaluate

pred = 'i love u'
label = 'love u'

metric = evaluate.load("rouge")
result = metric.compute(predictions=pred, references=label,  use_stemmer=True)
result

ValueError: Mismatch in the number of predictions (8) and references (6)

In [9]:
import evaluate
from nltk.tokenize import word_tokenize

pred = ['日本号　は　脆い']
label = ['毛おい　は　ない']
metric = evaluate.load("rouge")
result = metric.compute(predictions=pred, references=label, use_stemmer=True, tokenizer=word_tokenize)
print(result)


{'rouge1': 0.3333333333333333, 'rouge2': 0.0, 'rougeL': 0.3333333333333333, 'rougeLsum': 0.3333333333333333}


In [ ]:
from nltk.tokenize import word_tokenize
import evaluate
from transformers import AutoTokenizer
metric = evaluate.load('rouge')

tokenizer = AutoTokenizer.from_pretrained('ku-nlp/bart-base-japanese')

preds = ['中国']
labels = ['日本　語']

scores = metric.compute(
    predictions=preds,
    references=labels,
    tokenizer=tokenizer,
)

In [10]:
from datasets import load_dataset

raw_datasets = load_dataset(
    "json",
    data_dir="datasets/pubmed-dataset-incremental",
)

raw_datasets

DatasetDict({
    train: Dataset({
        features: ['article_id', 'article_text', 'abstract_text', 'section_names', 'sections'],
        num_rows: 24816
    })
    validation: Dataset({
        features: ['article_id', 'article_text', 'abstract_text', 'section_names', 'sections'],
        num_rows: 1393
    })
    test: Dataset({
        features: ['article_id', 'article_text', 'abstract_text', 'section_names', 'sections'],
        num_rows: 1430
    })
})

In [2]:
max_n_segments = 2

def truncate_max_n_segments(example):
    example['sections'] = example['sections'][:max_n_segments]
    example['abstract_text'] = example['abstract_text'][:max_n_segments]
    return example

for split in raw_datasets.keys():
    raw_datasets[split] = raw_datasets[split].map(truncate_max_n_segments)

NameError: name 'raw_datasets' is not defined

In [6]:
raw_datasets['test']['sections'][0]

['ohss is a serious complication of ovulation induction , occurring in 1 - 10% of in vitro fertilization patients  ( rizk and aboulghar , 1991 ; brinsden et al . , 1995 ) . this iatrogenic condition has a spectrum of clinical and laboratory manifestations ranging from mild to severe , even life - threatening conditions  ( rizk et al . , 1990 ) . among the serious manifestations of ohss are ascites and pleural effusion  ( rizk and smitz , 1992 ) . vascular endothelial growth factor has emerged as one of the factors most likely involved in the pathophysiology of ohss  ( geva and jaffe , 2000 ; rizk and aboulghar , 2010 ) . vascular endothelial growth factor is an angiogenic cytokine that is a potent stimulator of the vascular endothelium and appears to play an integral role in follicular growth , corpus luteum function and ovarian angiogenesis  ( rizk et al . , 1997 ) . vascular endothelial growth factor causes increased capillary permeability with its sequelae of ascites and possible pl

In [8]:
len(raw_datasets['test']['sections'][0])

2

In [13]:
max_n_segments = 11

def extend_max_n_segments(example):
    # num_segments_to_copy = max_n_segments - len(example['sections'])
    origin_segments_num = len(example['sections'])
    num_segments_to_copy = max_n_segments - origin_segments_num

    while num_segments_to_copy > 0 :
        segments_to_copy_temp = min(origin_segments_num, num_segments_to_copy)
        
        example['sections'] += example['sections'][:segments_to_copy_temp]
        example['abstract_text'] += example['abstract_text'][:segments_to_copy_temp]
        
        num_segments_to_copy -= segments_to_copy_temp
        
    return example


for split in raw_datasets.keys():
    raw_datasets[split] = raw_datasets[split].map(extend_max_n_segments)

Map: 100%|██████████| 1430/1430 [00:00<00:00, 5416.69 examples/s]


In [14]:
raw_datasets['test']['sections'][0]

['ohss is a serious complication of ovulation induction , occurring in 1 - 10% of in vitro fertilization patients  ( rizk and aboulghar , 1991 ; brinsden et al . , 1995 ) . this iatrogenic condition has a spectrum of clinical and laboratory manifestations ranging from mild to severe , even life - threatening conditions  ( rizk et al . , 1990 ) . among the serious manifestations of ohss are ascites and pleural effusion  ( rizk and smitz , 1992 ) . vascular endothelial growth factor has emerged as one of the factors most likely involved in the pathophysiology of ohss  ( geva and jaffe , 2000 ; rizk and aboulghar , 2010 ) . vascular endothelial growth factor is an angiogenic cytokine that is a potent stimulator of the vascular endothelium and appears to play an integral role in follicular growth , corpus luteum function and ovarian angiogenesis  ( rizk et al . , 1997 ) . vascular endothelial growth factor causes increased capillary permeability with its sequelae of ascites and possible pl